In [ ]:
from PIL import Image, ImageDraw
import numpy as np

# Culcurate file size (Change the numerical value arbitrarily)
def filter_size(w, h, x, y):
    size = 0
    if y < h * 2 / 3 or y > h * 7 /11 or x < w / 3 or x > w * 7 / 11:
        size = 1
    if y < h / 4 or y > h * 8 / 11 or x < w / 4 or x > w * 8 / 11:
        size = 2
    if y < h / 5 or y > h * 9 / 11 or x < w / 5 or x > w * 9 / 11:
        size = 3
    return size

# Return an image with averaging filter applied
def unclear_filter(img, w, h):
    image_pixcels = np.array([[img.getpixel((x,y)) for x in range(w)] for y in range(h)])
    filtered_img = Image.new('RGB', (w, h))
    for x in range(w):
        for y in range(h):
            # Filter step by step
            s = filter_size(w, h, x, y)
            if s != 0:
                x1 = max(0, x - s)
                x2 = min(x + s, w - 1)
                y1 = max(0, y - s)
                y2 = min(y + s, h - 1)
                x_size = x2 - x1 + 1
                y_size = y2 - y1 + 1
                mean_r, mean_g, mean_b = image_pixcels[y1:y2 + 1, x1:x2 + 1].reshape(x_size * y_size, 3).mean(axis = 0)
            else:
                mean_r, mean_g, mean_b = image_pixcels[y][x]
            filtered_img.putpixel((x, y), (int(mean_r), int(mean_g), int(mean_b)))
    return filtered_img

# Returns saturated image
def clear_filter(img, w, h):
    k1 = 1.9
    k2 = -0.4
    image_pixcels = np.array([[img.getpixel((x, y)) for x in range(w)] for y in range(h)])
    filtered_img = Image.new('RGB', (w, h))
    # Convert all pixels in the image
    for x in range(w):
        for y in range(h):
            r, g, b = image_pixcels[y][x]
            fr = int(min(255, max(0, r * k1 + g * k2 + b * k2)))
            fg = int(min(255, max(0, r * k2 + g * k1 + b * k2)))
            fb = int(min(255, max(0, r * k2 + g * k2 + b * k1)))
            filtered_img.putpixel((x, y), (fr, fg, fb))
    return filtered_img

# Read image
filename = "hoge.jpg"
img = Image.open("./" + filename).convert("RGB")
w, h = img.size

# Increase saturation of the Image
clear_img = clear_filter(img, w, h)

# Blur up and down
unclear_img = unclear_filter(clear_img, w, h)

# Save the image
unclear_img.save("./mini_" + filename)

